# Imports

In [3]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

## test model

In [11]:
url = "https://images.unsplash.com/photo-1748244487783-5ad28948a7c3?q=80&w=1938&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


{'scores': tensor([0.9997], grad_fn=<IndexBackward0>), 'labels': tensor([1]), 'boxes': tensor([[ 529.4136, 1095.3047, 1500.0292, 2539.8669]],
       grad_fn=<IndexBackward0>)}
Detected person with confidence 1.0 at location [529.41, 1095.3, 1500.03, 2539.87]


In [14]:
print(image.size)

(1938, 2568)


## Explication des coordonnées des boîtes englobantes

Les valeurs dans `results["boxes"]` représentent les coordonnées des boîtes englobantes (bounding boxes) qui délimitent les objets détectés dans l'image.

**Format des coordonnées :** `[x_min, y_min, x_max, y_max]`

- **x_min** : Position horizontale du coin supérieur gauche de la boîte (en pixels)
- **y_min** : Position verticale du coin supérieur gauche de la boîte (en pixels)
- **x_max** : Position horizontale du coin inférieur droit de la boîte (en pixels)
- **y_max** : Position verticale du coin inférieur droit de la boîte (en pixels)

**Exemple actuel :** `[529.41, 1095.30, 1500.03, 2539.87]`
- La boîte commence à la position (529, 1095) dans l'image
- Elle se termine à la position (1500, 2540)
- Largeur de la boîte : 1500 - 529 = 971 pixels
- Hauteur de la boîte : 2540 - 1095 = 1445 pixels

Ces coordonnées sont dans le système de coordonnées de l'image originale (1938 x 2568 pixels).

{0: 'N/A', 1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 12: 'N/A', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 26: 'N/A', 27: 'backpack', 28: 'umbrella', 29: 'N/A', 30: 'N/A', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 45: 'N/A', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 66: 'N/A', 67: 'dining table', 68: 'N/A', 69: 'N/A', 70: 'toilet